In [141]:
import cv2
import glob
import numpy as np
DATA_DIR = "./input"

def read_list_from_file(list_file, comment='#', func=None):
    with open(list_file) as f:
        lines  = f.readlines()

    strings=[]
    for line in lines:
        s = line.split(comment, 1)[0].strip()
        if s != '':
            strings.append(s)
    if func is not None:
        strings=[func(s) for s in strings]

    return strings
        
def run_make_one_mask():

    split = 'names'
    ids = read_list_from_file(DATA_DIR + '/names/' + split, comment='#')

    num_ids = len(ids)
    for i in range(num_ids):
        id = ids[i]
        image_files = glob.glob(DATA_DIR + '/' + id + '/images/*.png')

        assert(len(image_files)==1)
        image_file=image_files[0]
        image = cv2.imread(image_file,cv2.IMREAD_COLOR)

        H,W,C = image.shape
        one_mask = np.zeros((H,W), dtype=bool)
        mask_files =glob.glob(DATA_DIR + '/' + id + '/masks/*.png')
        for mask_file in mask_files:
            mask = cv2.imread(mask_file,cv2.IMREAD_GRAYSCALE)
            one_mask = one_mask |(mask>128)

        one_mask = (one_mask*255).astype(np.uint8)
        cv2.imwrite(DATA_DIR + '/' + id + '/one_mask.png', one_mask)


In [142]:
run_make_one_mask()

In [148]:
from torch.utils.data.dataset import Dataset

class ScienceDataset(Dataset):

    def __init__(self, split, mode='train', transform= None):
        super(ScienceDataset, self).__init__()
        self.split = split
        self.mode = mode
        self.transform = transform
        ids = read_list_from_file(DATA_DIR + '/names/' + split, comment='#')
        self.ids = ids

    def __getitem__(self, index):
        id   = self.ids[index]
        name = id.split('/')[-1]
        image_file = DATA_DIR + '/' + id + '/images/' + name +'.png'
        image = cv2.imread(image_file, cv2.IMREAD_COLOR)

        if self.mode in ['train']:
            mask_file =  DATA_DIR + '/' + id + '/one_mask.png'
            mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
            return self.transform(image,mask,index)

        if self.mode in ['test']:
            return self.transform(image,index)

    def __len__(self):
        return len(self.ids)

In [149]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *
def image_show(name, image, resize=1):
    H,W = image.shape[0:2]
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, image.astype(np.uint8))
    cv2.resizeWindow(name, round(resize*W), round(resize*H))

In [150]:
WIDTH = 256
HIGHT =256
def augment(image,mask,index):
        image = cv2.resize(image,(WIDTH,HIGHT))
        image = image.transpose(2, 1, 0)
        mask  = cv2.resize(mask,(WIDTH,HIGHT))
        mask  = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY)[1]
        mask[mask == 255.] = 1
        return image.astype(float),mask.astype(float),index

dataset = ScienceDataset(
        'names', mode='train', transform = augment
    )
sampler = RandomSampler(dataset)

In [146]:
import torch.nn as nn
def UpDownConv(in_channels, mid_channels, out_channels):
    return nn.Sequential(
            *ConvRelu(in_channels, mid_channels, kernel_size=3, stride=1, padding=1 ),
            *ConvRelu(mid_channels, out_channels, kernel_size=3, stride=1, padding=1 ),
    )
def ConvRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return [
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    ]
class UNet(nn.Module):
    
    def __init__(self, in_shape, num_classes):
        super(UNet, self).__init__()
        in_channels, height, width = in_shape

        self.down1 = nn.Sequential(
            *ConvRelu(in_channels, 16, kernel_size=3, stride=1, padding=1 ),
            *ConvRelu(16, 32, kernel_size=3, stride=2, padding=1 ),
        )
        #64
        self.down2 = UpDownConv(32, 64, 128)
        #32
        self.down3 = UpDownConv(128, 256, 512)
        #16
        self.down4 = UpDownConv(512, 512, 512)
        #8
        self.same = nn.Sequential(
            *ConvRelu(512,512, kernel_size=1, stride=1, padding=0 ),
        )
        #16
        self.up4 = UpDownConv(1024, 512, 512)
        #16
        self.up3 = UpDownConv(1024, 512, 128)
        #32
        self.up2 = UpDownConv(256, 128, 32)
        #64
        self.up1 = UpDownConv(64, 64, 32)
        #128

        self.up0 = nn.Sequential(
            *ConvRelu(32, 32, kernel_size=3, stride=1, padding=1 ),
        )
        #256

        self.classify = nn.Conv2d(32, num_classes, kernel_size=1, stride=1, padding=0 )

    def forward(self, x):

        down1 = self.down1(x)
        out   = F.max_pool2d(down1, kernel_size=2, stride=2) #64
        down2 = self.down2(out)
        out   = F.max_pool2d(down2, kernel_size=2, stride=2) #32
        down3 = self.down3(out)
        out   = F.max_pool2d(down3, kernel_size=2, stride=2) #16
        down4 = self.down4(out)
        out   = F.max_pool2d(down4, kernel_size=2, stride=2) # 8
        out   = self.same(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #16
        out   = torch.cat([down4, out],1)
        out   = self.up4(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #32
        out   = torch.cat([down3, out],1)
        out   = self.up3(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #64
        out   = torch.cat([down2, out],1)
        out   = self.up2(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #128
        out   = torch.cat([down1, out],1)
        out   = self.up1(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #256
        out   = self.up0(out)
        out   = self.classify(out)
        return out
    

In [147]:
num_classes = 1
batch_size  = 8
C = 3
from torch.autograd import Variable
import torch.nn.functional as F

inputs = torch.randn(batch_size,C,HIGHT,WIDTH)
labels = torch.LongTensor(batch_size,HIGHT,WIDTH).random_(1).type(torch.FloatTensor)

net = UNet(in_shape=(C,HIGHT,WIDTH), num_classes=1).train()
x = Variable(inputs)
y = Variable(labels)
logits = net.forward(x)

probs = F.sigmoid(logits)
probs_flat = probs.view (-1)
targets_flat = y.view(-1)

print('logits')
print(logits)
loss = nn.BCELoss()
output = loss(probs_flat, targets_flat)
output.backward()

print(type(net))
print(net)

logits
Variable containing:
( 0 , 0 ,.,.) = 
  8.4369e-02  3.4564e-02  9.0957e-02  ...   5.9912e-02  9.1005e-02  1.3273e-01
  1.0644e-02 -2.3685e-01 -7.3182e-02  ...  -4.6478e-02  1.3328e-01  1.8209e-02
 -1.6854e-01 -1.0098e-01  1.2093e-01  ...   1.0792e-01  7.6314e-02  1.1062e-01
                 ...                   ⋱                   ...                
 -9.6418e-03  1.6744e-02 -7.1006e-02  ...  -6.1589e-02 -1.4987e-01 -7.6147e-02
 -5.5725e-02 -5.0114e-02 -2.0890e-02  ...  -1.1394e-01 -1.3021e-01 -1.2679e-01
  2.0496e-01  9.4597e-02  1.2861e-01  ...   7.4939e-02  3.1585e-02 -6.9538e-02
      ⋮  

( 1 , 0 ,.,.) = 
  1.0872e-01 -5.3103e-02  3.3511e-02  ...  -3.9897e-02 -5.8542e-02  1.7054e-01
  3.9196e-02 -1.5089e-01 -7.9598e-02  ...  -1.5046e-01 -2.8465e-01  4.1013e-02
 -5.0152e-02 -1.5853e-01  2.0457e-04  ...  -8.7670e-02 -2.0622e-01  9.4710e-02
                 ...                   ⋱                   ...                
 -1.5737e-01 -3.2868e-02 -4.0399e-02  ...   1.2330e-01  1.

In [143]:
output

Variable containing:
 0.7930
[torch.FloatTensor of size 1]

In [122]:
import torch.optim as optim
LR = 0.0002
batch_size = 5
net = UNet256_3x3(in_shape=(C,HIGHT,WIDTH), num_classes=1).train()
epochs = 1
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
def train():
    optimizer = \
                optim.Adam(net.parameters(),
                           lr= LR, betas=(0.5, 0.999))
    train_dataset = ScienceDataset(
                                'names', mode='train',
                                transform = augment)
    train_loader  = DataLoader(
                        train_dataset,
                        sampler = RandomSampler(train_dataset),
                        batch_size  = batch_size,
                        drop_last   = True,
                        num_workers = 8)
    i = 0
    while  i < epochs:  
        sum_train_loss = 0.0
        sum_train_acc  = 0.0
        sum = 0
        net.train()
        optimizer.zero_grad()
        for index, data  in enumerate(train_loader):
            image,mask,index = data
            tensors = Variable(image)
            masks   = Variable(mask)
            logits  = net(tensors.float())
            probs   = F.sigmoid(logits).view(-1)
            masks_flat = masks.view(-1)
            print(probs.shape)
            print(masks_flat)
            loss = nn.BCELoss()
            output = loss(probs, masks_flat.float())
            output.backward()
            optimizer.step()
            optimizer.zero_grad()
            adjust_learning_rate(optimizer, i)
            batch_loss = output.data[0]
            sum_train_loss += batch_loss
            print('\r %0.2f| %0.4f| %0.4f  %0.4f  ',i, LR, batch_loss, sum_train_loss)

        i = i +1


In [123]:
train()

torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.7367463111877441 0.7367463111877441
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.7397809028625488 1.476527214050293
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.7235813140869141 2.200108528137207
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.7341080904006958 2.934216618537903
torch.Size([327680])
Variable containing:
 1
 1
 1
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.7359360456466675 3.6701526641845703
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6648772954940796 32.13082027435303
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.642602801322937 32.773423075675964
torch.Size([327680])
Variable containing:
 1
 1
 1
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6544080972671509 33.427831172943115
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6530956625938416 34.08092683553696
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6530814170837402 34.7340082526207
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6432571411132812 35.37726539373398
torch.Size([327680])
Variable containing:
 0

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6362491250038147 62.263804733753204
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 1
 1
 1
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6304605007171631 62.89426523447037
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 1
 1
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6308497190475464 63.525114953517914
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6141675114631653 64.13928246498108
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6312943696975708 64.77057683467865
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %6.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6257402896881104 65.39631712436676
torch.Size([327680])
Variable containing:


In [127]:
import torch 
torch.save(net.state_dict(), DATA_DIR +'/checkpoint/model.pth')